In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.io as pio

pio.renderers.default = 'browser'
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_curve, roc_auc_score, auc, ConfusionMatrixDisplay, confusion_matrix ,accuracy_score
import time
from sklearn.cluster import KMeans
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
from keras.datasets import imdb
from keras.layers import LSTM
from keras.layers import Embedding, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from sklearn.model_selection import train_test_split


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

start = time.time()
colnames = ['target', 'id', 'date', 'flag', 'user', 'text']
path = "/content/drive/MyDrive/sentiment140/Sentiment140.csv"
df = pd.read_csv(path, names=colnames,encoding='latin-1')
ma = df.head()

In [7]:
ma.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
################################################################################ EDA
df.shape  # (1599999, 6)
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [9]:
################################################################################ reodernamiento
pct10 = int(len(df) * .01)
pct_sample = int(pct10 * .1)

In [10]:
# split data into train y test
df_4 = df.query("target == 4").reset_index(drop=True).sample(n=pct10, replace=True, random_state=7).iloc[:, [0, -1]]
df_0 = df.query("target == 0").reset_index(drop=True).sample(n=pct10,replace=True, random_state=7).iloc[:,[0, -1]]  # (800000, 6)
print(len(df_4), len(df_0))
df_4.target = df_4.target.replace({4: 1})

df4_sample = df_4.sample(n=pct_sample, replace=True, random_state=7)  # 1600
df0_sample = df_0.sample(n=pct_sample, replace=True, random_state=7)  # 1600
print(len(df4_sample), len(df0_sample))
df_40 = pd.concat([df4_sample, df0_sample], axis=0)  # 3200

y = df_40.target.replace({4: 1})

16000 16000
1600 1600


In [11]:
# ---------------------------------Data preprocessing

# --- Data cleaning
# all tweets in lower case and no punctuation marks and
# removing of insignificant words in the observations, stopwords.

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
# stemming is the practice of involves reducing a words to its root form
# steeming helps to reduce sparcity degree of the data
ps = PorterStemmer()

stemmed_dataset = []
for i in range(0, len(df_40)):
    stemmed_array = df_40['text'].iloc[i].split()
    stemmed = [ps.stem(word) for word in stemmed_array if not word in set(stopwords.words('english'))]
    stemmed = ' '.join(stemmed)
    stemmed_dataset.append(stemmed)

In [13]:
# Fit the CountVectorizer to the training data
'''
The count vectorizer just counts the instance/frequency of a word in an
entire observation.'''

cv = CountVectorizer(lowercase=True, stop_words='english').fit(stemmed_dataset)
X = cv.fit_transform(stemmed_dataset)

In [14]:
#-------------------------------------------------------split traintest

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train=X_train.toarray()
X_test=X_test.toarray()


In [15]:
# --------------------------------------------------------DATA MODEL
from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier(n_neighbors=2)
classifier_knn.fit(X_train, y_train)
y_pred_knn = classifier_knn.predict(X_test)


from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred_log = clf_log.predict(X_test)

In [16]:

# -------------------------------CF
cf_test_knn = confusion_matrix(y_test, y_pred_knn)
acc_knn = accuracy_score(y_test, y_pred_knn)
print(cf_test_knn)
print(acc_knn)

cf_test_log = confusion_matrix(y_test, y_pred_log)
print(cf_test_log)
acc_log= accuracy_score(y_test, y_pred_log)
print(acc_log)

[[478  31]
 [490  57]]
0.5066287878787878
[[368 141]
 [188 359]]
0.6884469696969697
